In [1]:
import joblib
import pyodbc
import pandas as pd
#import numpy as np
#import matplotlib.pyplot as plt
#from ydata_profiling import ProfileReport
#import math
#from sklearn.metrics import roc_auc_score
#from scipy import stats
#import random
#from tqdm import tqdm
#import json

In [3]:
df = pd.read_csv('09_18_23_Thrive_Whitelist.csv')

In [4]:
df = df.replace(r'\r+|\n+|\t+','', regex=True)
df.columns = df.columns.str.replace(r'\r+|\n+|\t+','', regex=True)

In [8]:
df.columns

Index(['amplitude_id', 'user_id', 'White List Date ', 'carrier', 'city',
       'country', 'device', 'device_type', 'dma', 'expiry_updated',
       ...
       'gp:swift_combine_networking_enabled', 'gp:thrive_marketing_analytics',
       'gp:userId', 'gp:userType', 'gp:user_profitability_version',
       'gp:user_profitability_version_enabled', 'gp:version_management',
       'gp:vipStatus', 'gp:virtual_card_cvv_enabled',
       'gp:virtual_card_enabled'],
      dtype='object', length=120)

In [9]:
df[['user_id', 'White List Date ']]

,user_id,White List Date
0,ae25adf6-aed4-47f5-bd37-0b6602efd1f9,9/18/23
1,7c16972c-a1e8-4c0e-82c3-525f017b29fd,9/18/23
2,6bdf6125-7a60-403c-8fba-05fb836048d7,9/18/23
3,27e39e0f-4a69-49e0-803d-81b4a098c1a8,9/18/23
4,0caf18d1-29b4-4ae1-a49a-2046b42fc414,9/18/23
...,...,...
4674,53429aba-3570-483f-ac2d-74febacb70fe,9/18/23
4675,0a9ac8ad-dd52-41dc-90f3-dd4308e4f1d3,9/18/23
4676,11873303-59d8-4d7f-95a1-6ae34fd55753,9/18/23
4677,5c32f976-85ef-4052-a143-b5deac59c766,9/18/23


In [10]:
list_user_tokens = df['user_id'].to_list()

In [11]:
formatted_tokens = "', '".join(list_user_tokens)

In [12]:
formatted_tokens

"ae25adf6-aed4-47f5-bd37-0b6602efd1f9', '7c16972c-a1e8-4c0e-82c3-525f017b29fd', '6bdf6125-7a60-403c-8fba-05fb836048d7', '27e39e0f-4a69-49e0-803d-81b4a098c1a8', '0caf18d1-29b4-4ae1-a49a-2046b42fc414', '72145b20-b097-46ff-8c85-42a8ffe969bc', '1f589468-ae7a-4107-b9ca-7bb1bb459998', 'ab222771-8b3d-44b6-bd81-724fe9cc1a80', '559fd421-806d-4231-813c-609e88e92f99', 'ce513505-9d0a-4fba-a97f-3ba914b8bf78', '727fe936-5fe1-461b-8515-10f6fae0b956', 'db719b16-b309-4694-9816-487d4f46370b', 'f73229f6-b7c9-42d5-9d4e-4bfbed1c29de', '282950eb-73bf-406a-9978-62e5f71d804e', '07bd1a36-4171-4d96-b754-9611a271df5b', 'c85784c9-8810-4a9e-8218-884774656d3c', '38787bd9-7cc0-4f62-8d93-1542a8d6e674', '3faf2656-a7eb-4941-9c8c-ea7dcd6ff2e7', '2cea07eb-5dd8-46b4-bffc-2ade14b8bb98', '74edab2d-512d-4e39-9815-76eec4929d3a', 'c6395d01-45e7-49cf-831a-ed81dab2952b', '7650f09f-39c0-4acc-b84c-f0f825e89cc6', 'ca13174c-c005-4a6f-bc8a-14dbef9a4d99', '0181d4fa-34f8-48d3-8639-88a7329c6b89', '192917ac-6833-41d7-b559-ca9ccbb2694d', 

In [18]:
sql_query = f""" 
select a.UserId, 
a.PublicToken,
b.userloanapplicationid,
b.STATUS,
b.createdat,
case when status = 0 then 'Denied'
when status = 1 then 'Created'
when status = 2 then 'Incomplete'
when status = 3 then 'Qualified'
when status = 4 then 'Accepted' 
else 'Error'
end as Status_text,
d.[Rule],
uca.MLModelScore as Aries_score
from [User] a 
left join UserLoanApplications b
on a.userid = b.UserId
left join UserLoanApplicationDenialReasons as d 
on b.userLoanApplicationId = d.userLoanApplicationId
left join UserCreditModel as ucm 
on b.UserCreditModelId = ucm.UserCreditModelId
left join UserCreditvariableAries as uca
on ucm.UserCreditVariableId = uca.UserCreditVariableId
where PublicToken in ('{formatted_tokens}')
"""

In [19]:
import struct
import pyodbc
import subprocess
import pandas as pd
from azure.identity import AzureCliCredential

# sign in via the Azure CLI 'az login' command.
create_app = subprocess.run('az login', shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
print(create_app.stdout.decode("utf-8"))

# use the cli credential to get a DB token after signing in via the Azure CLI 'az login' command.
credential = AzureCliCredential()
db_token = credential.get_token('https://database.windows.net/')
expires_in = pd.to_datetime(db_token[1], unit='s').tz_localize('UTC') - pd.Timestamp('now', tz='utc')
print(f'DB token expires in {round(expires_in.seconds / 60)} minutes')

tokenb = bytes(db_token[0], "UTF-16-LE")
tokenstruct = struct.pack("=i", len(tokenb)) + tokenb


# build connection string using acquired token
server = 'sql-empower-prod-uscentral.database.windows.net'
database = 'empower-prod2'
conn_string = f"Driver=ODBC Driver 17 for SQL Server;SERVER={server};DATABASE={database}"
SQL_COPT_SS_ACCESS_TOKEN = 1256
conn = pyodbc.connect(conn_string, attrs_before = {SQL_COPT_SS_ACCESS_TOKEN:tokenstruct})

df_db = pd.read_sql(sql_query, conn)
print(df_db)
conn.close()

[
  {
    "cloudName": "AzureCloud",
    "homeTenantId": "84a204fb-1e53-4fbb-9d40-dc6b242a352b",
    "id": "69219f8b-3126-4f06-9ac2-59ef68d2e009",
    "isDefault": true,
    "managedByTenants": [
      {
        "tenantId": "2f4a9838-26b7-47ee-be60-ccc1fdec5953"
      }
    ],
    "name": "Visual Studio Enterprise: BizSpark",
    "state": "Enabled",
    "tenantId": "84a204fb-1e53-4fbb-9d40-dc6b242a352b",
    "user": {
      "name": "aditya@empower.me",
      "type": "user"
    }
  },
  {
    "cloudName": "AzureCloud",
    "homeTenantId": "84a204fb-1e53-4fbb-9d40-dc6b242a352b",
    "id": "8d4e2597-1862-4357-b76b-d710c3ff4dae",
    "isDefault": false,
    "managedByTenants": [],
    "name": "Pay-As-You-Go Dev/Test",
    "state": "Enabled",
    "tenantId": "84a204fb-1e53-4fbb-9d40-dc6b242a352b",
    "user": {
      "name": "aditya@empower.me",
      "type": "user"
    }
  }
]

DB token expires in 58 minutes


/var/folders/vf/t7s9kyz57vb4wyk1cvrhc6pc0000gn/T/ipykernel_27895/3398815910.py:28: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_db = pd.read_sql(sql_query, conn)


       UserId                           PublicToken  userloanapplicationid  \
0     7925074  98DA8105-0550-4A79-8A02-E7B5BB6E4299                87557.0   
1     7347034  90F9219D-A71F-44E5-9E0D-C7344B9D5605                87620.0   
2     7925558  3D731460-BC94-4675-84B5-AAD49D4BD2A8                87624.0   
3     7925548  FF8238EE-C9FD-4DCC-8CE7-746B0DFABEF2                87627.0   
4     7652701  4663795C-DBF2-4A81-B258-43DCC5F461B8                87632.0   
...       ...                                   ...                    ...   
4688  7520695  0FB2E752-4EDB-4922-943F-5BAF8605C133                87927.0   
4689  7714873  B8E08E72-A2BB-46CA-B327-5DA037655FE9                88465.0   
4690  7517446  56B4F053-FF52-4963-91C0-202494938F81                88093.0   
4691  7866163  039655BE-4CED-4A50-8E5D-17D0D08ED7DB                87551.0   
4692  7655806  AD659D8D-C5D0-46F8-B858-AACD9A0F710D                88342.0   

      STATUS                  createdat Status_text  Rule  Arie

In [20]:
df_db.to_csv('September_Expansion_checks.csv')

In [25]:
sql_query_Pre_qual = f""" 
select a.*
from(
select
usr.UserId as UserId_wl,
b.score as aries_prequal_score,
e.DateGenerated as aries_prequal_date,
row_number() over(partition by usr.Userid order by b.score ASC) as aries_prequal_row_num
from  [User] usr
left join UserCreditVariableLithium as c
on usr.UserId = c.UserId
and ActualAnalysisDateUtc >= '2023-09-16'
left join UserCreditVariableMLScore as b
on c.UserCreditVariableId = b.UserCreditVariableId
left join UserCreditvariableAriesPreQual e 
on c.UserCreditVariableId = e.UserCreditVariableId
where b.ModelType = 5
and usr.PublicToken in ('{formatted_tokens}')) A
where aries_prequal_row_num = 1
"""

In [26]:
import struct
import pyodbc
import subprocess
import pandas as pd
from azure.identity import AzureCliCredential

# sign in via the Azure CLI 'az login' command.
create_app = subprocess.run('az login', shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
print(create_app.stdout.decode("utf-8"))

# use the cli credential to get a DB token after signing in via the Azure CLI 'az login' command.
credential = AzureCliCredential()
db_token = credential.get_token('https://database.windows.net/')
expires_in = pd.to_datetime(db_token[1], unit='s').tz_localize('UTC') - pd.Timestamp('now', tz='utc')
print(f'DB token expires in {round(expires_in.seconds / 60)} minutes')

tokenb = bytes(db_token[0], "UTF-16-LE")
tokenstruct = struct.pack("=i", len(tokenb)) + tokenb


# build connection string using acquired token
server = 'sql-empower-prod-uscentral.database.windows.net'
database = 'empower-prod2'
conn_string = f"Driver=ODBC Driver 17 for SQL Server;SERVER={server};DATABASE={database}"
SQL_COPT_SS_ACCESS_TOKEN = 1256
conn = pyodbc.connect(conn_string, attrs_before = {SQL_COPT_SS_ACCESS_TOKEN:tokenstruct})

df_db_pre_qual = pd.read_sql(sql_query_Pre_qual, conn)
print(df_db_pre_qual)
conn.close()

[
  {
    "cloudName": "AzureCloud",
    "homeTenantId": "84a204fb-1e53-4fbb-9d40-dc6b242a352b",
    "id": "69219f8b-3126-4f06-9ac2-59ef68d2e009",
    "isDefault": true,
    "managedByTenants": [
      {
        "tenantId": "2f4a9838-26b7-47ee-be60-ccc1fdec5953"
      }
    ],
    "name": "Visual Studio Enterprise: BizSpark",
    "state": "Enabled",
    "tenantId": "84a204fb-1e53-4fbb-9d40-dc6b242a352b",
    "user": {
      "name": "aditya@empower.me",
      "type": "user"
    }
  },
  {
    "cloudName": "AzureCloud",
    "homeTenantId": "84a204fb-1e53-4fbb-9d40-dc6b242a352b",
    "id": "8d4e2597-1862-4357-b76b-d710c3ff4dae",
    "isDefault": false,
    "managedByTenants": [],
    "name": "Pay-As-You-Go Dev/Test",
    "state": "Enabled",
    "tenantId": "84a204fb-1e53-4fbb-9d40-dc6b242a352b",
    "user": {
      "name": "aditya@empower.me",
      "type": "user"
    }
  }
]

DB token expires in 9 minutes


/var/folders/vf/t7s9kyz57vb4wyk1cvrhc6pc0000gn/T/ipykernel_27895/3450636912.py:28: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_db_pre_qual = pd.read_sql(sql_query_Pre_qual, conn)


      UserId_wl  aries_prequal_score         aries_prequal_date  \
0       7013657             0.146603 2023-09-18 11:28:09.996009   
1       7013936             0.148551 2023-09-16 12:17:18.589086   
2       7014195             0.138241 2023-09-16 05:29:20.302973   
3       7014249             0.140815 2023-09-18 12:38:40.453982   
4       7014435             0.114077 2023-09-16 17:09:14.409007   
...         ...                  ...                        ...   
4671    7842999             0.109382 2023-09-18 07:07:28.212522   
4672    7843140             0.148231 2023-09-18 09:58:15.759141   
4673    7843210             0.121737 2023-09-18 13:38:43.411136   
4674    7843549             0.139276 2023-09-18 08:46:17.739915   
4675    7843673             0.131634 2023-09-17 17:59:02.743820   

      aries_prequal_row_num  
0                         1  
1                         1  
2                         1  
3                         1  
4                         1  
...            

In [29]:
df_db_pre_qual.to_csv('September_Expansion_checks_pre_qual.csv')